# MATH50003 Numerical Analysis: Problem 3

This problem sheet explores implementation of triangular solves,
supporting a matrix with two super-diagonals, as well as
permutation and Householder reflections that can be applied to a vector in
$O(n)$ complexity.

In [1]:
using LinearAlgebra, Test

# We will override these functions below
import Base: getindex, setindex!, size, *, \

## 1. Dense Matrices

**Problem 1.1** Show that `A*x` is not
implemented as `mul(A, x)` from the lecture notes
by finding a `Float64` example  where the bits do not match.

In [2]:
function mul(A, x)
    m,n = size(A)
    c = zeros(eltype(x), m) # eltype is the type of the elements of a vector/matrix
    for j = 1:n, k = 1:m
        c[k] += A[k, j] * x[j]
    end
    c
end

mul (generic function with 1 method)

In [3]:
# Problem 1.1
function test_mul(n=2, times=100)
    result = 0
    for j = 1:times
        v = randn(n)
        A = randn(n,n)
        if A * v != mul(A, v)
            result = (v, A, A*v==mul(A,v))
            break
        end
    end
    result
end
test_mul()

([0.2699132219603334, 2.232230867523209], [1.3922478634060933 -1.1091273492239513; 0.7261640441448557 -0.9312389033067757], false)

## 2. Triangular Matrices

**Problem 2.1** Complete the following functions for solving linear systems with
triangular systems by implementing back and forward-substitution:

In [4]:
function ldiv1(U::UpperTriangular, b)
    n = size(U,1)

    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using back-substitution
    x[n] = b[n] / U[n,n]

    for i = (n-1):-1:1
        x[i] = (b[i] - dot(x,U[i,:])) / U[i,i]
    end
    x
end

function ldiv2(U::LowerTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using forward-substitution
    x[1] = b[1] / U[1,1]
    for i = 2:n
        x[i] = (b[i] - dot(x,U[i,:])) / U[i,i]
    end
    x
end

ldiv2 (generic function with 1 method)

In [5]:
# Testing implementation
using LinearAlgebra
A = [1 2 3;
     4 5 6;
     7 8 9]
U = UpperTriangular(A)
L = LowerTriangular(A)

b = [12, 22, 18]
c = [2, 18 , 48]

# ldiv1(U, b)
ldiv2(L, c .* 1000000)

3-element Vector{Float64}:
 2.0e6
 2.0e6
 2.0e6

**Problem 2.2⋆** Given $𝐱 ∈ ℝ^n$, find a lower triangular matrix of the form
$$
L = I - 2 𝐯 𝐞_1^⊤
$$
such that:
$$
L 𝐱 = x_1 𝐞_1.
$$
What does $L𝐲$ equal if $𝐲  ∈ ℝ^n$ satisfies $y_1 = 𝐞_1^⊤ 𝐲 = 0$?

## 3. Banded matrices

**Problem 3.1** Complete the implementation of `UpperTridiagonal` which represents a banded matrix with
bandwidths $(l,u) = (0,2)$:

In [85]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    # TODO: return U[k,j]
    result = 0
    n = length(U.d)
    
    if j-k ==2
        result = du2[k]
    elseif j-k==1
        result = du[k]
    elseif k==j
        result = d[k]
    else
        result = 0
    end
    result
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j > k+2
        error("Cannot modify off-band")
    end

    # TODO: modify d,du,du2 so that U[k,j] == v
    if abs(k-j)==2
        du2[k] = v
    elseif abs(k-j)==1
        du[k] = v
    elseif k==j
        d[k] = v
    end
    U # by convention we return the matrix
end

setindex! (generic function with 91 methods)

Test implementation

In [87]:
UTD = UpperTridiagonal(Vector([1,2,3,4]), Vector([5,6,8]), Vector([11,12]))
# 1 5 11 0
# 0 2 6  12
# 0 0 3  8
# 0 0 0  4

# size(UTD,1)
setindex!(UTD, 11451, 2, 2)
@test getindex(UTD, 2, 2) == 11451
UTD

4×4 UpperTridiagonal{Int64}:
 1      5  11   0
 0  11451   6  12
 0      0   3   8
 0      0   0   4

**Problem 3.2** Complete the following implementations of `*` and `\` for `UpperTridiagonal` so that
they take only $O(n)$ operations.

In [88]:
function *(U::UpperTridiagonal, x::AbstractVector)
    T = promote_type(eltype(U), eltype(x)) # make a type that contains both the element type of U and x
    b = zeros(T, size(U,1)) # returned vector
    # TODO: populate b so that U*x == b (up to rounding)
    for i in 1:size(U,1)
        b[i] = U.d * x[i] + U.du * x[i+1] + U.du2 * x[i+2]
    end
    b   
end

function \(U::UpperTridiagonal, b::AbstractVector)
    T = promote_type(eltype(U), eltype(b)) # make a type that contains both the element type of U and b
    x = zeros(T, size(U,2)) # returned vector
    # TODO: populate x so that U*x == b (up to rounding)
    d,du,du2 = U.d,U.du,U.du2
    m = size(U, 1)
    n = size(U, 2)

    @assert m==n # what is the case for non-square matrices

    x[n] = b[n] / d[n]
    x[n-1] = (b[n-1] - x[n] * du[n-1]) / d[n-1]
    
    for i in (n-2):-1:1
        x[i] = (b[i] - x[i+2] * du2[i] - x[i+1] * du[i]) / d[i]
    end
    x
end

\ (generic function with 136 methods)

In [92]:
A = Matrix{Float64}([1 5 11 0; 0 2 6 12; 0 0 3 8; 0 0 0 4])
x = Vector{Float64}([0,1,1,1])
A\x
UTD = UpperTridiagonal(Vector([1,2,3,4]), Vector([5,6,8]), Vector([11,12]))
@test UTD \ x ≈ A\x

Test Passed
  Expression: UTD \ x ≈ A \ x
   Evaluated: [3.6666666666666665, 0.0, -0.3333333333333333, 0.25] ≈ [3.666666666666667, -5.551115123125783e-17, -0.3333333333333333, 0.25]

## 4. Permutations

**Problem 4.1⋆** What are the permutation matrices corresponding to the following permutations?
$$
\begin{pmatrix}
1 & 2 & 3 \\
3 & 2 & 1
\end{pmatrix}, \begin{pmatrix}
1 & 2 & 3 & 4 & 5 & 6\\
2 & 1 & 4 & 3 & 6 & 5
\end{pmatrix}.
$$


**Problem 4.2** Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` and such that `*` takes $O(n)$ operations.

In [55]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        # Checking validity of construction
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p) # Special function available to inner constructors which created a new object of the type
    end
end

size(P::PermutationMatrix) = (length(P.p),length(P.p))

function getindex(P::PermutationMatrix, k::Int, j::Int)
    # TODO: Return P[k,j]
    if j==P.p[k]
        1
    else
        0
    end
end

function *(P::PermutationMatrix, x::AbstractVector)
    # TODO: permute the entries of x
    n = size(P,1)
    new = zeros(Int, n)
    for i in 1:n
        new[i] = x[P.p[i]]
    end
    new
end

# If your code is correct, this "unit test" will succeed
p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]
# Other tests
q = [1,2,3,4,5]
@test P*q == [1,4,2,5,3]

Test Passed
  Expression: P * q == [1, 4, 2, 5, 3]
   Evaluated: [1, 4, 2, 5, 3] == [1, 4, 2, 5, 3]

## 5. Orthogonal matrices

**Problem 5.1⋆** Show that orthogonal matrices preserve the 2-norm of vectors:
$$
\|Q 𝐯\| = \|𝐯\|.
$$


**Problem 5.2⋆** Show that the eigenvalues $λ$ of an orthogonal matrix $Q$ are
on the unit circle: $|λ| = 1$.


**Problem 5.3⋆** Explain why an orthogonal matrix $Q$ must be equal to $I$ if all its eigenvalues are 1.


**Problem 5.4** Complete the implementation of a type representing
reflections that supports `Q[k,j]` and such that `*` takes $O(n)$ operations.

In [78]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

size(Q::Reflection) = (length(Q.v),length(Q.v))

function getindex(Q::Reflection, k::Int, j::Int)
    # TODO: Return Q[k,j]
    if k==j
        1 - 2*Q.v[k]*Q.v[j]/norm(Q.v)^2
    else
        -2*Q.v[k]*Q.v[j]/norm(Q.v)^2
    end
end

function *(P::Reflection, x::AbstractVector)
    # TODO: permute the entries of x
    T = promote_type(eltype(P), eltype(x))
    n = size(P, 1)
    new = zeros(T, n)
    for i in 1:n
        new[i] = x[i] - dot(P.v, x) * 2 * P.v[i]
    end
    new
end

# If your code is correct, these "unit tests" will succeed
x = randn(5)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q*v ≈ -v
@test Q'Q ≈ I


Test Passed
  Expression: Q' * Q ≈ I
   Evaluated: [1.0000000000000002 4.163336342344337e-17 … 1.1102230246251565e-16 -5.551115123125783e-17; 4.163336342344337e-17 1.0 … 0.0 2.7755575615628914e-17; … ; 1.1102230246251565e-16 0.0 … 1.0 0.0; -5.551115123125783e-17 2.7755575615628914e-17 … 0.0 1.0] ≈ UniformScaling{Bool}(true)

**Problem 5.5** Complete the following implementation of a Housholder reflection  so that the
unit tests pass.

In [79]:
function householderreflection(s::Bool, x::AbstractVector)
    # TODO: implement Householder reflection, returning a Reflection
    n = size(x, 1)
    e1 = zeros(n)
    e1[1] = 1
    if s
        y = -sign(x[1]) * norm(x) .* e1 + x
        Reflection(y)
    else
        y = sign(x[1]) * norm(x) .* e1 + x
        Reflection(y)
    end
end

x = randn(5)
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ≈ norm(x)*[1; zeros(length(x)-1)]

Q̃ = householderreflection(false, x)
@test Q̃ isa Reflection
@test Q̃*x ≈ -norm(x)*[1; zeros(length(x)-1)]

Test Passed
  Expression: Q̃ * x ≈ -(norm(x)) * [1; zeros(length(x) - 1)]
   Evaluated: [-2.2778736023760415, 0.0, 0.0, 0.0, 0.0] ≈ [-2.2778736023760415, -0.0, -0.0, -0.0, -0.0]